In [2]:
%load_ext autoreload
%autoreload 2

from cookie_monster_frontend_lib import create_df

STATUS_DIR = "/home/hpinkard_waller/GitRepos/EncodingInformation/led_array/phenotyping_experiments/config_files/"


df = create_df(STATUS_DIR, ('experiment name', 'status', 'date'))

df


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# import tensorflow as tf

# i = tf.compat.v1.train.summary_iterator('/home/hpinkard_waller/models/Different_input_contrasts_DPC_raw_trial1/tensorboard/train/events.out.tfevents.1677808652.waller-fuoco.1383916.0.v2')

# for e in i:
#     for v in e.summary.value:
#         if 'loss' in v.tag:
#             print(v.tag, tf.make_ndarray(v.tensor))

